In [ ]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim

tornado_tracks_df = pd.read_csv('../Datasets/Tornado_tracks_2015_2019.csv')
tornado_tracks_df.head()



,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_LOCATION,END_DATE,END_TIME,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER
0,558838,OLDHAM CO.,PEWEE VLY,4/2/2015,1400,Thunderstorm Wind,52,,0,0,...,PEWEE VLY,4/2/2015,1425,38.31,-85.49,38.31,-85.49,Broadcast media reported a large porch roof bl...,A stalled frontal boundary across the area bro...,1
1,558833,JEFFERSON CO.,LOUISVILLE,4/2/2015,1435,Hail,1.75,,0,0,...,LOUISVILLE,4/2/2015,1455,38.25,-85.74,38.25,-85.74,Golf ball size hail was reported by a trained ...,A stalled frontal boundary across the area bro...,2
2,558834,JEFFERSON CO.,INDIAN HILLS,4/2/2015,1440,Hail,1.25,,0,0,...,INDIAN HILLS,4/2/2015,1455,38.27,-85.70,38.27,-85.70,Half dollar size hail was reported by the public.,A stalled frontal boundary across the area bro...,3
3,558836,JEFFERSON CO.,INDIAN HILLS,4/2/2015,1445,Hail,0.88,,0,0,...,INDIAN HILLS,4/2/2015,1455,38.29,-85.64,38.29,-85.64,The public reported nickel size hail.,A stalled frontal boundary across the area bro...,4
4,558835,JEFFERSON CO.,ST MATTHEWS,4/2/2015,1445,Hail,1.75,,0,0,...,ST MATTHEWS,4/2/2015,1455,38.26,-85.64,38.26,-85.64,Golf ball size hail was reported by an amateur...,A stalled frontal boundary across the area bro...,5


In [4]:
tornado_tracks_df.shape

(291, 39)

In [5]:
tornado_tracks_df.columns

Index(['EVENT_ID', 'CZ_NAME_STR', 'BEGIN_LOCATION', 'BEGIN_DATE', 'BEGIN_TIME',
       'EVENT_TYPE', 'MAGNITUDE', 'TOR_F_SCALE', 'DEATHS_DIRECT',
       'INJURIES_DIRECT', 'DAMAGE_PROPERTY_NUM', 'DAMAGE_CROPS_NUM',
       'STATE_ABBR', 'CZ_TIMEZONE', 'MAGNITUDE_TYPE', 'EPISODE_ID', 'CZ_TYPE',
       'CZ_FIPS', 'WFO', 'INJURIES_INDIRECT', 'DEATHS_INDIRECT', 'SOURCE',
       'FLOOD_CAUSE', 'TOR_LENGTH', 'TOR_WIDTH', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'END_DATE',
       'END_TIME', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EVENT_NARRATIVE', 'EPISODE_NARRATIVE', 'ABSOLUTE_ROWNUMBER'],
      dtype='object')

In [6]:
tornado_tracks_df

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_LOCATION,END_DATE,END_TIME,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER
0,558838,OLDHAM CO.,PEWEE VLY,4/2/2015,1400,Thunderstorm Wind,52,,0,0,...,PEWEE VLY,4/2/2015,1425,38.3100,-85.4900,38.3100,-85.4900,Broadcast media reported a large porch roof bl...,A stalled frontal boundary across the area bro...,1
1,558833,JEFFERSON CO.,LOUISVILLE,4/2/2015,1435,Hail,1.75,,0,0,...,LOUISVILLE,4/2/2015,1455,38.2500,-85.7400,38.2500,-85.7400,Golf ball size hail was reported by a trained ...,A stalled frontal boundary across the area bro...,2
2,558834,JEFFERSON CO.,INDIAN HILLS,4/2/2015,1440,Hail,1.25,,0,0,...,INDIAN HILLS,4/2/2015,1455,38.2700,-85.7000,38.2700,-85.7000,Half dollar size hail was reported by the public.,A stalled frontal boundary across the area bro...,3
3,558836,JEFFERSON CO.,INDIAN HILLS,4/2/2015,1445,Hail,0.88,,0,0,...,INDIAN HILLS,4/2/2015,1455,38.2900,-85.6400,38.2900,-85.6400,The public reported nickel size hail.,A stalled frontal boundary across the area bro...,4
4,558835,JEFFERSON CO.,ST MATTHEWS,4/2/2015,1445,Hail,1.75,,0,0,...,ST MATTHEWS,4/2/2015,1455,38.2600,-85.6400,38.2600,-85.6400,Golf ball size hail was reported by an amateur...,A stalled frontal boundary across the area bro...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,841596,JEFFERSON CO.,RIVERSIDE GARDENS,6/23/2019,1541,Thunderstorm Wind,56,,0,0,...,RIVERSIDE GARDENS,6/23/2019,1541,38.1700,-85.8900,38.1700,-85.8900,ASOS reported a 56 mph wind gust at Louisville...,"On June 23, like the events on June 21, large ...",287
287,841574,JEFFERSON CO.,ST MATTHEWS,6/23/2019,1555,Thunderstorm Wind,52,,0,0,...,ST MATTHEWS,6/23/2019,1555,38.2500,-85.6500,38.2500,-85.6500,A large tree blew down.,"On June 23, like the events on June 21, large ...",288
288,841576,OLDHAM CO.,BUCKNER,6/23/2019,1600,Thunderstorm Wind,55,,0,0,...,BUCKNER,6/23/2019,1600,38.4000,-85.4400,38.4000,-85.4400,Numerous trees and power lines were down acros...,"On June 23, like the events on June 21, large ...",289
289,843731,OLDHAM CO.,FLOYDSBURG,7/16/2019,2005,Thunderstorm Wind,52,,0,0,...,FLOYDSBURG,7/16/2019,2005,38.3092,-85.4392,38.3092,-85.4392,Dispatch reported a tree down on Mount Zion Ro...,"On July 14th, as the remains of Hurricane Barr...",290


In [9]:
tornado_tracks_df.isnull().sum()

EVENT_ID                0
CZ_NAME_STR             0
BEGIN_LOCATION          0
BEGIN_DATE              0
BEGIN_TIME              0
EVENT_TYPE              0
MAGNITUDE               0
TOR_F_SCALE             0
DEATHS_DIRECT           0
INJURIES_DIRECT         0
DAMAGE_PROPERTY_NUM     0
DAMAGE_CROPS_NUM        0
STATE_ABBR              0
CZ_TIMEZONE             0
MAGNITUDE_TYPE          0
EPISODE_ID              0
CZ_TYPE                 0
CZ_FIPS                 0
WFO                     0
INJURIES_INDIRECT       0
DEATHS_INDIRECT         0
SOURCE                  0
FLOOD_CAUSE             0
TOR_LENGTH              0
TOR_WIDTH               0
BEGIN_RANGE             0
BEGIN_AZIMUTH           0
END_RANGE               0
END_AZIMUTH             0
END_LOCATION            0
END_DATE                0
END_TIME                0
BEGIN_LAT               0
BEGIN_LON               0
END_LAT                 0
END_LON                 0
EVENT_NARRATIVE        33
EPISODE_NARRATIVE       0
ABSOLUTE_ROW

In [31]:
geolocator = Nominatim(user_agent="tornado_project")
geolocator

In [28]:
tornado_tracks_df['Zip'] = 0

for i in range(len(tornado_tracks_df)):
    location = geolocator.reverse(str(tornado_tracks_df.loc[i, 'BEGIN_LAT']) + ', ' + str(tornado_tracks_df.loc[i, 'BEGIN_LON']),timeout=10)
    address = location.raw['address']
    tornado_tracks_df.loc[i, 'Zip'] = address.get('postcode')




C:\Users\15025\AppData\Local\Temp\ipykernel_13232\3275896752.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '40056' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tornado_tracks_df.loc[i, 'Zip'] = address.get('postcode')


In [32]:
tornado_tracks_df

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_DATE,END_TIME,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER,Zip
0,558838,OLDHAM CO.,PEWEE VLY,4/2/2015,1400,Thunderstorm Wind,52,,0,0,...,4/2/2015,1425,38.3100,-85.4900,38.3100,-85.4900,Broadcast media reported a large porch roof bl...,A stalled frontal boundary across the area bro...,1,40056
1,558833,JEFFERSON CO.,LOUISVILLE,4/2/2015,1435,Hail,1.75,,0,0,...,4/2/2015,1455,38.2500,-85.7400,38.2500,-85.7400,Golf ball size hail was reported by a trained ...,A stalled frontal boundary across the area bro...,2,40202
2,558834,JEFFERSON CO.,INDIAN HILLS,4/2/2015,1440,Hail,1.25,,0,0,...,4/2/2015,1455,38.2700,-85.7000,38.2700,-85.7000,Half dollar size hail was reported by the public.,A stalled frontal boundary across the area bro...,3,40206
3,558836,JEFFERSON CO.,INDIAN HILLS,4/2/2015,1445,Hail,0.88,,0,0,...,4/2/2015,1455,38.2900,-85.6400,38.2900,-85.6400,The public reported nickel size hail.,A stalled frontal boundary across the area bro...,4,40222
4,558835,JEFFERSON CO.,ST MATTHEWS,4/2/2015,1445,Hail,1.75,,0,0,...,4/2/2015,1455,38.2600,-85.6400,38.2600,-85.6400,Golf ball size hail was reported by an amateur...,A stalled frontal boundary across the area bro...,5,40207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,841596,JEFFERSON CO.,RIVERSIDE GARDENS,6/23/2019,1541,Thunderstorm Wind,56,,0,0,...,6/23/2019,1541,38.1700,-85.8900,38.1700,-85.8900,ASOS reported a 56 mph wind gust at Louisville...,"On June 23, like the events on June 21, large ...",287,40258
287,841574,JEFFERSON CO.,ST MATTHEWS,6/23/2019,1555,Thunderstorm Wind,52,,0,0,...,6/23/2019,1555,38.2500,-85.6500,38.2500,-85.6500,A large tree blew down.,"On June 23, like the events on June 21, large ...",288,40207
288,841576,OLDHAM CO.,BUCKNER,6/23/2019,1600,Thunderstorm Wind,55,,0,0,...,6/23/2019,1600,38.4000,-85.4400,38.4000,-85.4400,Numerous trees and power lines were down acros...,"On June 23, like the events on June 21, large ...",289,40031
289,843731,OLDHAM CO.,FLOYDSBURG,7/16/2019,2005,Thunderstorm Wind,52,,0,0,...,7/16/2019,2005,38.3092,-85.4392,38.3092,-85.4392,Dispatch reported a tree down on Mount Zion Ro...,"On July 14th, as the remains of Hurricane Barr...",290,40014


In [33]:
tornado_tracks_df['Zip'].unique()

array(['40056', '40202', '40206', '40222', '40207', '40242', '40041',
       '40118', '40299', '40243', '40220', '40014', '40258', '40165',
       '40213', '40272', '40229', '40129', '40218', '40023', '40209',
       '40245', '40026', '40150', '40211', '40047', '40241', '40223',
       '40291', '40205', '40215', '40059', '40217', '40216', '40109',
       '40204', '40210', '40031', '40214', '40212', None, '40219',
       '40228', '40208', '40177', '40203', '40221', '40010', '40071'],
      dtype=object)